In [1]:
!pip install -q crewai langchain langchain_google-genai langchain_community langchain_core langchain_mistralai duckduckgo-search wikipedia langchain_experimental arxiv faiss-cpu langchainhub pypdf gpt4all

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.0/51.0 kB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 809.1/809.1 kB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 52.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 260.9/260.9 kB 19.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.6/177.6 kB 15.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 31.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 286.1/286.1 kB 22.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.1/4.1 MB 42.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 257.5/257.5 kB 19.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.4/58.4 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.7/105.7 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.0/68.0 kB 7.0 

In [2]:
from crewai import Agent , Task , Crew , Process
import getpass
import os
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_experimental.tools import PythonREPLTool
from langchain.tools import DuckDuckGoSearchRun ,WikipediaQueryRun
from langchain.agents import Tool
from langchain_experimental.utilities import PythonREPL
from langchain_community.utilities import WikipediaAPIWrapper
import subprocess
import duckduckgo_search
import pprint
from langchain.retrievers import ArxivRetriever
from langchain_community.document_loaders import TextLoader
from langchain.chains import create_retrieval_chain

In [3]:
from google.colab import userdata
os.environ["GOOGLE_API_KEY"] = userdata.get("GOOGLE_API_KEY")

In [4]:
from langchain.pydantic_v1 import BaseModel, Field
from langchain.tools import BaseTool, StructuredTool, tool
@tool
def toolmaker(a):
    """Look up things online."""
    with open("test.py", "w") as f:
      f.write(a)
random_tool= Tool(
    name="random",
    description="Use must use this tool only whenever you to create new langchaintools that you might need in the future.You have to send the code to make the tool",
    func=toolmaker,
)

In [5]:
py_tool=PythonREPLTool()
search_tool=DuckDuckGoSearchRun()
# retriever = ArxivRetriever(load_max_docs=2)
# docs = retriever.get_relevant_documents(query="1605.08386")
# loader = TextLoader("/content/untitled.txt")
# documents = loader.load()
# from langchain.text_splitter import CharacterTextSplitter
# from langchain_community.vectorstores import FAISS
# from langchain_google_genai import GoogleGenerativeAIEmbeddings
# from langchain_community.embeddings import GPT4AllEmbeddings
# text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
# texts = text_splitter.split_documents(documents)
# embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
# db = FAISS.from_documents(texts, embeddings)
# embeddings=GPT4AllEmbeddings()
# db=FAISS.load_local("/content/drive/FAISS",embeddings)
# retriever = db.as_retriever()
# from langchain.tools.retriever import create_retriever_tool

# retriever_tool = create_retriever_tool(
#     retriever,
#     "langchain_agents",
#     "Searches and returns data about langchain agents.",
# )
tools = [py_tool,random_tool,search_tool]#retriever_tool,retriever_tool,toolmaker,search_tool,search_tool

In [6]:
from langchain import hub

prompt = hub.pull("hwchase17/react")

In [7]:
model=ChatGoogleGenerativeAI(model="gemini-pro", convert_system_message_to_human=True)

In [8]:
from langchain.agents import AgentExecutor, create_openai_tools_agent , initialize_agent ,create_react_agent
agent = create_react_agent(model, tools, prompt)

In [9]:
# Create an agent executor by passing in the agent and tools
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True,handle_parsing_errors=True)

In [10]:
agent_executor.invoke(
    {
        "input": "create a text file and store hi in it "
    }
)



> Entering new AgentExecutor chain...


Action: Python_REPL
Action Input: `f = open("hi.txt", "w")`Action: Python_REPL
Action Input: `f.write("hi")`Action: Python_REPL
Action Input: `f.close()`Final Answer: None

> Finished chain.


{'input': 'create a text file and store hi in it ', 'output': 'None'}

In [11]:
!pip install gradio -U -q
import gradio as gr

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.0/17.0 MB 61.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.1/92.1 kB 12.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.7/310.7 kB 32.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 27.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 kB 7.9 MB/s eta 0:00:00


In [14]:
def ask(question):
  return qa.invoke(question)['result']
iface = gr.Interface(fn=ask, inputs=gr.Textbox(
    value="Why should you choose RMK over other colleges"),
        outputs="markdown",
        title="Test Site",
        description="Ask a question our RMK college.",
        examples=[['What are the departments are there in RMK'],['What is so special about E.C.E'],['Why R.M.K is the best College'],['Tell me more about the placement in RMK']],
    theme=gr.themes.Soft(),
    allow_flagging="never",)

iface.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://b9f93ae4670daf5fde.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
